In [2]:
import requests 
import lxml.html as lh
import pandas as pd

In [3]:
#we used  crawling with help from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059 website
url='https://www.google.com/search?q=pm2.5+levels&oq=p&aqs=chrome.0.69i59l2j69i57j69i59j35i39j0i433i512j46i199i433i465i512j46i131i433j0i131i433j0i433i512.767j0j15&sourceid=chrome&ie=UTF-8'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]


[2, 2, 2, 2, 2]

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"PM2.5"
2:"Air Quality Index"


In [33]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 2, the //tr data is not from our table 
    if len(T)!=2:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [34]:
[len(C) for (title,C) in col]


[4, 4]

In [35]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df

,PM2.5,Air Quality Index
0,0 to 12.0,Good 0 to 50
1,12.1 to 35.4,Moderate 51 to 100
2,35.5 to 55.4,Unhealthy for Sensitive Groups 101 to 150
3,55.5 to 150.4,Unhealthy 151 to 200


In [36]:
df.to_csv("pm25.csv")

In [38]:
df_no2=pd.read_csv("no_two.csv")
df_no2

,Nitrogen dioxide(averaged over 1 hpur,Air quality
0,less than 60,Good
1,60-120,Fair
2,120-180,Poor
3,180-360,Very poor
4,over 360,Extremely poor


In [40]:
pm_col=df["PM2.5"]

In [42]:
df_no2['pm2.5']=pm_col
df_no2

,Nitrogen dioxide(averaged over 1 hpur,Air quality,pm2.5
0,less than 60,Good,0 to 12.0
1,60-120,Fair,12.1 to 35.4
2,120-180,Poor,35.5 to 55.4
3,180-360,Very poor,55.5 to 150.4
4,over 360,Extremely poor,NaN


In [43]:
df_no2.fillna("-")

,Nitrogen dioxide(averaged over 1 hpur,Air quality,pm2.5
0,less than 60,Good,0 to 12.0
1,60-120,Fair,12.1 to 35.4
2,120-180,Poor,35.5 to 55.4
3,180-360,Very poor,55.5 to 150.4
4,over 360,Extremely poor,-


In [45]:
df_no2.to_csv("Toxic.csv")